In [10]:
import pandas as pd
data = pd.read_csv('train.csv', header=None, skiprows=1, parse_dates=[0,11,12])
print("finished reading train.csv")
data.columns = ["date_time", "site_name", "posa_continent", "user_location_country",
                "user_location_region", "user_location_city", "orig_destination_distance",
                "user_id", "is_mobile", "is_package", "channel", "srch_ci", "srch_co",
                "srch_adults_cnt", "srch_children_cnt", "srch_rm_cnt", "srch_destination_id",
                "srch_destination_type_id", "is_booking", "cnt", "hotel_continent", "hotel_country",
                "hotel_market", "hotel_cluster"] 
data.drop(data.index[[406250,1373714,1373715,2374706,3504146,3707429,7780825,778086,8040327,8428261,8428262,
                     15336804,16428469,16695744,17669511,18156670,18575457,31775817,31784369,32146102,32168215,32549132,
                     34815966,34815968,34815969,35412891,36606126,36833435,36833436,37330211]], inplace=True)
#data = data.loc[data['is_booking'] == 1] # contains only booking events
#print(data.shape)

finished reading train.csv


In [11]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 16 21:32:03 2016

@author: Michael
"""

import pandas as pd
import numpy as np
"from ml_metrics import average_precision"


print("training data loaded")
#dest = pd.read_csv('destinations_mod.csv')
#print("dest data loaded")
test = pd.read_csv('test.csv', parse_dates=['date_time', 'srch_co'])
print("test data loaded")
test = test.drop('id', 1)
   
data = data.drop('cnt', 1)
data = data.drop('is_booking', 1)
data = data.drop('site_name', 1)
data = data.drop('posa_continent', 1)
data = data.drop('is_mobile', 1)
data = data.drop('is_package', 1)
data = data.drop('srch_adults_cnt', 1)
data = data.drop('srch_children_cnt', 1)
data = data.drop('srch_rm_cnt', 1)
data = data.drop('srch_destination_type_id', 1)
data = data.drop('hotel_continent', 1)
data_y = data["hotel_cluster"]
data = data.drop('hotel_cluster', 1)
data['srch_ci'] = pd.to_datetime(data['srch_ci'], errors='coerce')
data['srch_ci'].fillna(value=pd.to_datetime('7/1/2014'), inplace=True)
data['srch_co'] = pd.to_datetime(data['srch_co'], errors='coerce')
data['srch_co'].fillna(value=pd.to_datetime('7/4/2014'), inplace=True)
data['orig_destination_distance'].fillna(1969, inplace=True)
test = test.drop('site_name', 1)
test = test.drop('posa_continent', 1)
test = test.drop('is_mobile', 1)
test = test.drop('is_package', 1)
test = test.drop('srch_adults_cnt', 1)
test = test.drop('srch_children_cnt', 1)
test = test.drop('srch_rm_cnt', 1)
test = test.drop('srch_destination_type_id', 1)
test = test.drop('hotel_continent', 1)
test['srch_ci'].replace(to_replace='nan', value=pd.to_datetime('1/1/2017'), inplace=True)
test.set_value(312920, 'srch_ci', pd.to_datetime('1/21/2016'))
test['srch_ci'] = pd.to_datetime(test['srch_ci'], errors='coerce')
test['srch_co'] = pd.to_datetime(test['srch_co'], errors='coerce')
test['srch_ci'].fillna(value=pd.to_datetime('7/1/2015'), inplace=True)
test['srch_co'].fillna(value=pd.to_datetime('7/3/2015'), inplace=True)
test['orig_destination_distance'].fillna(1685, inplace=True)
test_bookings = test.as_matrix(['srch_ci','srch_co'])
train_bookings = data.as_matrix(['srch_ci','srch_co'])



print("pp part 1 complete")

training data loaded
test data loaded
pp part 1 complete


In [12]:
from datetime import date
import re
def addStayLengths(matrix_input): #takes in numpy matrices and returns 
    length_of_stay_values = []
    for i in range(0, matrix_input.shape[0]):
        m1 = re.search(r'(\d+-\d+-\d+)', str(matrix_input[i][0])) #srch_ci
        m2 = re.search(r'(\d+-\d+-\d+)', str(matrix_input[i][1])) #srch_co
        match1 = m1.group(0)
        values1 = match1.split('-')
        year1 = int(values1[0])
        month1 = int(values1[1])
        day1 = int(values1[2]) 

        match2 = m2.group(0)
        values2 = match2.split('-')         
        year2 = int(values2[0])
        month2 = int(values2[1])
        day2 = int(values2[2])     
    #try:
        d0 = date(year1, month1, day1)
        d1 = date(year2, month2, day2)
        delta = d1 - d0
        length_of_stay_values.append(delta.days)
    #except:
    #    print("exception: " + str(matrix_input[i][0]) + ", " + str(matrix_input[i][1]))
    #    length_of_stay_values.append(3)
    return length_of_stay_values
traincolumn = addStayLengths(train_bookings)
testcolumn = addStayLengths(test_bookings)
data = data.assign(length_of_stay = traincolumn)
test = test.assign(length_of_stay = testcolumn)
print("length of stay values appended")


length of stay values appended


In [13]:
def addFeatures(df_input):
    df_input['date_time_month'] = df_input['date_time'].dt.month
    df_input['date_time_day'] = df_input['date_time'].dt.day
    df_input['date_time_hour'] = df_input['date_time'].dt.hour
    df_input['srch_ci_month'] = df_input['srch_ci'].dt.month
    df_input['srch_ci_dayofweek'] = df_input['srch_ci'].dt.dayofweek
    df_input['srch_ci_day'] = df_input['srch_ci'].dt.day
    #df_input['srch_co_month'] = df_input['srch_co'].dt.month
    print("new features added")
    df_input = df_input.drop('date_time', 1)
    df_input = df_input.drop('srch_ci', 1)
    df_input = df_input.drop('srch_co', 1)
    #df_input = pd.merge(df_input, dest, on='srch_destination_id', how='left')
    #df_input.fillna(-2.18, inplace=True)
    #df_input = df_input.drop('srch_destination_id', 1)
    print("pp part 2 complete")
    return df_input
    
data = addFeatures(data)
test = addFeatures(test)

new features added
pp part 2 complete
new features added
pp part 2 complete


In [14]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaest = [2,5,10,15]
data2 = data.iloc[:int(data.shape[0]/2)][:]
data_y2 = data_y.iloc[:int(data_y.shape[0]/2)][:]
for ada in adaest:
    clf = AdaBoostClassifier(n_estimators=ada)
    clf.fit(data2.values, data_y2.values)
    probs = clf.predict_proba(test.values)
    pred = [];
    indices = [];
    for i in range(len(test)):
        pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
        indices.append(i)  

    df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    df.to_csv("result_12_8_16_Adaboost_" + str(ada) + ".csv", index=False)

In [ ]:
#MLP
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_6_16_MLP.csv", index=False)

In [ ]:
#GNB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_6_16_GNB.csv", index=False)

In [ ]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis() #singular value decomposition (default)
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_6_16_LDA.csv", index=False)

In [ ]:
#QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_6_16_QDA.csv", index=False)

In [ ]:
#SGD - logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log',n_jobs=-1)
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_6_16_SGD_log.csv", index=False)

In [ ]:
#SGD - SVM-like?
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='modified_huber',n_jobs=-1) #sort of like SVM?
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_6_16_SGD_modified_huber.csv", index=False)

In [ ]:
#DT Gini
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=25)
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_7_16_DT_gini_20.csv", index=False)

In [ ]:
#DT Entropy
clf = DecisionTreeClassifier(criterion="entropy", max_depth=25)
clf.fit(data.values, data_y.values)
probs = clf.predict_proba(test.values)
pred = [];
indices = [];
for i in range(len(test.values)):
    pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
    indices.append(i)  

df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv("result_12_7_16_DT_entropy_20.csv", index=False)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
est = [15, 20, 25, 30]
for e in est:
    clf = RandomForestClassifier(criterion = "entropy", n_estimators=e, max_depth=20,warm_start=True)
    clf.fit(data.values, data_y.values)
    probs = clf.predict_proba(test.values)

    pred = [];
    indices = [];
    for i in range(len(test)):
        pred.append(" ".join(map(str, list(reversed(probs[i].argsort()[-5:])))))
        indices.append(i)  

    df = pd.DataFrame({"id" : indices, "hotel_cluster" : pred})
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    df.to_csv("result_12_7_16_RF_20depth" + str(e) + "est_2014.csv", index=False)
